# On Calculating Upper Bounds of ReLU Singular Values
If you use this resource in a scientific publication, we would appreciate references to the following paper: TODO


## What are we doing?
We aim to calculate upper bounds of data dependent ReLU singular values.

## What is a ReLU singular value?
Let $A\in\mathbb{R}^{m\times n}$ and $b\in\mathbb{R}^m$, we then define the $k$th ReLU singular value of the operator $\text{ReLU}(A\cdot+b)$ over the set $X\subset\mathbb{R}^m$ as

$s_{k, X} = \min_{\text{rank}(L)\le k}\max_{x\in X} ||\text{ReLU}(Ax+b)-\text{ReLU}(Lx+b)||_2.$


## Approach/Algorithm
The calculation is a two step process:

1. Approximate

    $W_\ast, M_\ast = \displaystyle\text{argmin}_{\substack{W \in \mathbb{R}^{m\times k}\\M\in \mathbb{R}^{k \times n}}} \sum_{x\in X} ||\text{ReLU}(Ax + b)-\text{ReLU}(WMx + b)||_2^2$,
   
    e.g. via some kind of stochastic gradient descent (in our case Adam).
2. Calculate the approximate upper bound via

    $s_{k, X} \le \max_{x\in X}||\text{ReLU}(Ax + b)-\text{ReLU}(W_\ast M_\ast x + b)||_2$.

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
def fit_relu_layer_low_rank(A, b, xs_train, rank, batch_size=-1, log_loss_every_n_batches=100):
    """
    Calculates a low rank approximation of rank k of the ReLU layer ReLU(Ax+b)
    w.r.t. the data set xs_train. I.e. an approximation of the form ReLU(WMx+b),
    where (for A of the shape m x n) W of the shape m x rank and
    M of the shape rank x n.

    Keyword arguments:
    A -- The weight matrix of the layer.
    b -- The bias of the layer.
    xs_train -- The data set w.r.t. which we calculate the ReLU singular value, given as a matrix where each column represents a data point.
    rank -- The rank of the approximation.
    batch_size -- The batch size with which we optimize for the approximation. -1 means: batch = full data set.
    log_loss_every_n_batches -- How often do we evaluate our progress?
    """
    # Set the batch size used for the optimization.
    if batch_size == -1:
        batch_size = xs_train.shape[1]

    # Calculate output of the layer to train approximation based on it.
    ys_train = np.clip(np.dot(A, xs_train) + b, 0, np.inf)

    # Calculate inital guess for the low rank approximation via a truncated SVD.
    W, d, M = np.linalg.svd(A)
    d = np.sqrt(d)
    for i in range(len(d)):
        W[:,i] *= d[i]
        M[i,:] *= d[i]
    rank = np.min([rank, len(d)])
    M = M[:rank,:]
    W = W[:,:rank]

    # Define Tensorflow variables for the approximation.
    W_tf = tf.Variable(W, dtype=tf.float64)
    M_tf = tf.Variable(M, dtype=tf.float64)
    b_tf = tf.Variable(b, dtype=tf.float64, trainable=False)

    # Define in- and output placeholders for training.
    xs = tf.placeholder(dtype=tf.float64)
    ys = tf.placeholder(dtype=tf.float64)

    # Define computational structure, i.e. the one layer "neural net".
    ys_predict = tf.nn.relu(tf.matmul(tf.matmul(W_tf, M_tf), xs) + b_tf)

    # Define loss-function: simple least squares
    loss = tf.reduce_sum((ys-ys_predict)**2)

    # Define optimizer 
    optimizer = tf.train.AdamOptimizer()
    train_step = optimizer.minimize(loss)
    
    # Close possibly still active TF sessions and start an interactive one.
    tf.InteractiveSession().close()
    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()

    # Training/Optimizing for the approximation.
    loss_s = []
    i = -1
    while True:
        i += 1
        # Create batch.
        idxs = (np.random.rand(batch_size)*xs_train.shape[1]).astype(int)
        batch_xs = xs_train[:,idxs]
        batch_ys = ys_train[:,idxs]

        # Make training step
        sess.run(train_step, feed_dict={xs: batch_xs, ys:batch_ys})

        # Check whether it's time to log our loss/progress.
        if i % log_loss_every_n_batches == 0:
            loss_train = sess.run(loss, feed_dict={xs:xs_train, ys:ys_train})
            loss_s += [np.sqrt(loss_train/xs_train.shape[1])]

        # If the current approximation is the best one yet, save it.
        if np.min(loss_s) == loss_s[-1]:
            W_best, M_best = sess.run(W_tf), sess.run(M_tf)

        # Well, this was pointless.
        if rank == 0:
            break

        # Heuristic to determine whether it makes sense to stop optimizing:
        # Every beginning is hard -- keep going.
        if i > log_loss_every_n_batches*10:
            # Have we struggled lately to improve?
            struggled = np.sum((np.array(loss_s[-11:-1])-np.array(loss_s[-10:]))>0) < 5 # Less than 5 times decreased the loss over the last 10 iterations?
            # And are we no longer in peek shape?
            not_peek_shape = np.min(loss_s) <= loss_s[-1]
            # If we struggle to improve and are no longer in peek shape, despied the fact that we have tried for a while, than maybe it's time to quit.
            if struggled and not_peek_shape:
                break

    # Get and log final loss.
    loss_train = sess.run(loss, feed_dict={xs:xs_train, ys:ys_train})
    loss_s += [np.sqrt(loss_train/xs_train.shape[1])]

    # Close Tensorflow session.
    tf.reset_default_graph()
    sess.close()

    return (W_best, M_best), loss_s

def relu_singular_value_via_approximation(A, b, xs, W, M):
    """
    Calculates an upper bound to ReLU singular value w.r.t. the data set xs
    of the layer ReLU(Ax+b) based on the low rank approximation ReLU(WMx+b).

    Keyword arguments:
    A -- The weight matrix of the layer.
    b -- The bias of the layer.
    xs -- The data set w.r.t. which we calculate the ReLU singular value.
    W -- First of the matrices producing the low rank approximation / bottleneck.
    M -- Second of the matrices producing the low rank approximation / bottleneck.
    """
    # calculate output of layer
    output_layer = np.clip(np.dot(A,xs)+b, 0, np.inf)
    # calculate output of the low rank approximation to the layer
    output_approximation = np.clip(np.dot(np.dot(W,M),xs)+b, 0, np.inf)
    # calulate and return upper bound of ReLU singular value
    return np.max( np.linalg.norm(output_layer - output_approximation, axis=0) )

def relu_singular_value(A, b, xs, k, batch_size=-1, log_loss_every_n_batches=100):
    """
    Calculates an upper bound of the k-th ReLU singular value w.r.t. the data set xs
    of the layer ReLU(Ax+b).

    Keyword arguments:
    A -- The weight matrix of the layer.
    b -- The bias of the layer.
    xs -- The data set w.r.t. which we calculate the ReLU singular value.
    k -- Index of singular value, starting at 0!
    batch_size -- The batch size with which we optimize for the underlying approximation. -1 means: batch = full data set.
    log_loss_every_n_batches -- How often do we evaluate our progress in the underlying approximation?
    """
    # Get low rank approximation.
    (W, M), _ = fit_relu_layer_low_rank(A, b, xs, k, batch_size=batch_size, log_loss_every_n_batches=log_loss_every_n_batches)
    # calulate and return upper bound of ReLU singular value
    return relu_singular_value_via_approximation(A, b, xs, W, M)

# Simple Example
As an example we want to calculate an upper bound for the $k$th ReLU singular value of the layer $\text{ReLU}(Ax+b)$ over the dataset $X$.

Here $A\in\mathbb{R}^{n\times n}$ with entries sampled from $\mathcal{N}(0,1_{n})$, $b=0$ and $X$ is given by $N$ random vectors with entries sampled from the uniform distribution over the interval $[0, 1]$.

Once for $n = 10$, $k = 5$ and $N=1,000$ and once for $n = 50$, $k = 20$ and $N=100$.

In [3]:
A = np.random.normal(0, 1, [10, 10])
b = np.zeros([10, 1])
xs = np.random.rand(10, 10000)

bound_relu_singular_value_5 = relu_singular_value(A, b, xs, 5, log_loss_every_n_batches=10)
print(f"The 5th ReLU singular value of the layer ReLU(Ax+b), w.r.t. the data set X, is at most {bound_relu_singular_value_5}.")

The 5th ReLU singular value of the layer ReLU(Ax+b), w.r.t. the data set X, is at most 1.31384284081892.


In [4]:
A = np.random.normal(0, 1, [50, 50])
b = np.zeros([50, 1])
xs = np.random.rand(50, 100)

bound_relu_singular_value_20 = relu_singular_value(A, b, xs, 20, log_loss_every_n_batches=10)
print(f"The 20th ReLU singular value of the layer ReLU(Ax+b), w.r.t. the data set X, is at most {bound_relu_singular_value_20}.")

The 20th ReLU singular value of the layer ReLU(Ax+b), w.r.t. the data set X, is at most 3.8578114116420457.
